In [42]:
import copy
import warnings
import numpy as np 
import pandas as pd
from typing import List
import matplotlib.pyplot as plt
from numpy.random import choice
from scipy.cluster.hierarchy import linkage
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from scipy.cluster.hierarchy import dendrogram, linkage,to_tree
from collections import defaultdict
from scipy.cluster.hierarchy import ClusterNode
import warnings
import random
from tqdm import tqdm
warnings.filterwarnings('ignore')

In [6]:
# do not edit this cell
# load the data files (download from the LMS)
embedded_images = np.load('images.npy')
labels = np.load('labels.npy')

# split into pool & testing
X_pool, X_test, y_pool, y_test = train_test_split(embedded_images, labels, 
                                                  test_size=0.5, random_state=1234, shuffle=True)

# sample a seed set
np.random.seed(1234)
label2id = defaultdict(list)
for i, label in enumerate(y_pool):
    label2id[label].append(i)
seed_set = []
for label, ids in label2id.items():
    seed_set.extend(np.random.choice(ids, size=10, replace=False))

In [7]:
labelset, y_label = np.unique(y_pool, return_inverse=True)

In [8]:
mylinkage = linkage(X_pool,method="average")

In [48]:
def Cluster_adaptive_active_learning(X,y,max_size,mylinkage,beta):
    P_list = []
    #X_linkage = linkage(X,method='average')
    Tree = myHS(X,y,mylinkage)
    P = np.array([Tree.rootnode.id])
    ######################
    queries = []

    pbar = tqdm(total=max_size)
    while len(queries) < max_size:
        v = Tree.select(P)    
        z = Tree.rand_pick(v,queries)
        queries.append(z.id)
        label = y[z.id] 
  
        Tree.empirical_counts_proba(z,v,label)
        Tree.update_admissible(beta)
        Tree.update_score()
        P = Tree.update_L_P(P)
        P_list.append(P)
        pbar.update(1)
    #print(P_list)
    pbar.close()
    return queries, P_list

In [50]:
Query, P = Cluster_adaptive_active_learning(X_pool,y_label,5,mylinkage,2)

100%|██████████| 5/5 [00:01<00:00,  3.05it/s]


In [51]:
class myHS(object):
    """
    Construct a Hierarchical sampling class.

    """
    def __init__(self,X,y,linkage):
        """
        Initialize the Hierarchical sampling.
        Given the training set X and its Label y, 
        return the all the implementation methods.

        X: Training Data, size n * d
        y: The label of the Training Data, size n
        """
        keys = set(y)

        # Compute the Hierarchical tree for X
        #mylinkage = linkage(X,method="average")
        #print("Finish build likage..")
        # Build the Hierarchical Tree Structure
        self.rootnode,self.nodelist  = to_tree(mylinkage,rd = True)

        # Add Parent node to each node, earier for future implementation
        self.rootnode.parent = None
        for node in self.nodelist:
            if node.get_left():
                node.left.parent = node
            if node.get_right():
                node.right.parent = node

        # Record the number of class n
        self.n_labels = len(set(y))
        # the number of node in the tree
        self.n_nodes = len(self.nodelist)


        ## Update Empirical count and probabilities
        # Record the number of point sampled from node V of each class
        self.label_count = np.zeros((len(self.nodelist),self.n_labels))
        # Record the fraction of label l in points sampled from Subtree V
        self.proba = np.zeros((len(self.nodelist),self.n_labels))
                
        # Record the number of sampled time
        self.n = np.zeros(len(self.nodelist))


        ## Update Admissible and Score
        # Record if the node is admissible or not
        self.admissible = np.array(np.array([False]*len(self.nodelist)*self.n_labels).reshape(len(self.nodelist),self.n_labels))

        # Record score for each node
        self.score = np.zeros(len(self.nodelist))


        ## Prune and assign Labels
        # Record if achieve score/shoud it be pruned
        self.prune = np.zeros(len(self.nodelist))
        # Record the labels
        self.label = np.zeros(len(self.nodelist))


        # Record the Lower bound and Upper Bound
        self.upper = np.zeros((len(self.nodelist),self.n_labels))
        self.lower = np.zeros((len(self.nodelist),self.n_labels))    
   
    def select(self,P,selection_type="Upper"):
        """
        Selection Function, use Active Sampling as selection crieria.
        Return a selected node.

        P: Current pruning of tree, size p * 1 (A list of node index)
        selection_type: A string, indicates use Upper bound or Lower
        bound as the selection criteria. Default is Upper.
        """

        # Choose use Upper Bound or use Lower Bound
        if selection_type == "Upper":
            bound = self.upper[P]
        else:
            bound = self.lower[P]

        L = self.label[P].astype(int)
        weight = np.array([self.nodelist[node].count for node in P])
        bound = bound[np.arange(len(bound)),L]
        count = weight *(1-bound)
        return random.choices(P,weights = count/sum(count))[0]
    

    def leaves(self, node,leaf_list = None):
        """
        Get all the leaf node from a given node recursively. 
        Return a list of leaf node.

        node: Current root node
        leaf_list: A list used to store all the found leaf node.
        """
        node = self.nodelist[node]
        if not leaf_list:
            leaf_list = []
        if node.is_leaf():
            leaf_list.append(node)
            return leaf_list
        else:
            leaf_list = self.leaves(node.left.id,leaf_list)
            leaf_list = self.leaves(node.right.id,leaf_list)
            
            return leaf_list

    def rand_pick(self,node,queries,leaf_list=None):
        """
        Query node from the list of leaf node of the given subtree
        randomly. If a node has been queryed, then it will random
        select one node from the queries list.
        Return a selected node z

        node: Current root node
        queries: A list that record all queryed node
        leaf_list: A list used to store all the found leaf node.
        """
        
        leaves_list = self.leaves(node)
        leaves_id = np.array([leaf.id for leaf in leaves_list])
        queries = np.array(queries)

        not_query = np.setdiff1d(leaves_id,queries)
        if len(not_query) == 0:
            return self.nodelist[random.choice(queries)]
        else:
            return random.choice(leaves_list)


    def empirical_counts_proba(self,z,v,label):
        """
        Update the number of points queryed from node v and the
        fraction of label.

        z: Current node
        v: subroot node
        label: queryed z's label
        """

        if not isinstance(z,ClusterNode):
            z = self.nodelist[z]
        if not isinstance(v,ClusterNode):
            v = self.nodelist[v] 
 
        # make sure we are count all the leaf node under the subroot node v
        while z and z.id <= v.id:
            # Use Label count to record the label of each queryed node
            self.label_count[z.id][label]+=1
            # Record the number that the node has been counted
            self.n[z.id] +=1
            z = z.parent

        self.n = self.n.reshape((len(self.n), 1))      

        self.proba = self.label_count/self.n 


    def update_admissible(self,beta = 2):
        """
        Update the admissible (Node, label) pair. 
        The admissible is matrix that contains True and False.

        beta: parameter Beta with default value:2 
        """
        # Calculate the confidence interval of node v with label l
        delta = 1/self.n + np.sqrt(self.proba*(1-self.proba)/self.n)
        lower = np.fmax(self.proba-delta,0)
        upper = np.fmin(self.proba+delta,1)
        
        # Lemma 1
        for l in np.arange(self.n_labels):
            upper_lp = np.delete(upper,l,axis = 1)
            self.admissible[:,l] = np.all((1-lower[:,l][:,None])<beta*(1-upper_lp),axis = 1)
        
        # Empeirical estimate of the error
        self.E_vl = 1-self.proba
        self.E_vl[~self.admissible] = 1
        
        # Record the Upper Bound and Lower Bound to calculate select function
        self.upper = upper
        self.lower = lower
    

    def update_score(self):
        """
        Pruning the labeling of T_v achieving score. The score is use
        to select a good pruning
        """
        
        # Traversal all the node of the tree 
        for i in range(len(self.nodelist)):
            node = self.nodelist[i]

            # if a node is a leaf node, 
            # estimated error of pruning is used
            # leaf node should not be pruned
            if node.is_leaf():
                self.score[i] = np.nanmin(self.E_vl[i])
                self.prune[i] = False
            else:
                # For admissible pair:
                # check the estimated error and the estimated error 
                # of its children if the children's score is smaller, 
                # then the node should be pruned
                if self.admissible[i,:].any():
                    
                    a_score = (node.left.count/node.count) * self.score[node.left.id]
                    b_score = (node.right.count/node.count) * self.score[current_node.right.id]
                    if a_score+b_score < np.nanmin(self.E_vl[i]):
                        self.score[i] = a_score+b_score
                        self.prune[i] = True
                    else:
                        self.score[i] = np.nanmin(self.E_vl[i])
                        self.prune[i] = False
                else:
                    # For non-admissible pair, simplely use the score
                    self.score[i] = np.nanmin(self.E_vl[i])
                    self.prune[i] = False

    def recursively_update_label(self,node,P):
        """
        Recursively update the label of each node in current purne P

        node: Node from P
        P: Node to show the label 
        """

        if not isinstance(node, ClusterNode):
            node = self.nodelist[node]
        if not isinstance(P, ClusterNode):
            P = self.nodelist[P]

        if node.is_leaf():
            self.label[node.id] = self.label[P.id]
        else:
            self.recursively_update_label(node.left,P)
            self.recursively_update_label(node.right,P)

    def update_L_P(self,P):
        """
        Update the new pruning list P_prime, using the record                    self.prune from pervious steps.
        return the new pruned list P_prime 

        P: current Prune
        """
        
        P_prime = []
        for i in P:
            node = self.nodelist[i]

            # if the node is recorded to be pruned,
            # than it will be pruned and left its children nodes
            if self.prune[i]:

                label = np.where(self.admissible[i,:])[0][0]
                P_prime.append(node.left.id)
                P_prime.append(node.right.id)

                self.label[node.left.id] = label
                self.label[node.right.id] = label
            else:
                P_prime.append(i)
                self.label[node.id] = np.nanargmin(self.E_vl[i])
        # Assign all thelable for the P_prime
        # and update its labels       
        for p in P_prime:
            self.recursively_update_label(p,p)
        return P_prime
    